In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import pandas as pd
import numpy as np
import os
from common import *
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
#from excited_inception_v3 import SEInception3
#from inception_v3 import Inception3
#from xception import Xception
from resnet101 import ResNet101
from tqdm import tqdm_notebook
from pytorch_img_functions import ten_crop, resize
from PIL import Image

@common.py:  
	set random seed
		SEED=1513097643
	set cuda environment
		torch.cuda.device_count()   = 2
		torch.cuda.current_device() = 0



In [2]:
FILELIST_CSV ="/media/cvpr/SSD/Cdiscount/pytorch_models/release/demo_code/FileList.csv"


CATEGORIES_CSV="/media/cvpr/SSD/Cdiscount/pytorch_models/release/demo_code/categories.csv" 
category_ids = np.array(pd.read_csv(CATEGORIES_CSV, header=None).values).flatten()

#CATEGORIES_PKL='/media/cvpr/SSD/Cdiscount/downloaded/class_order.pkl'
#category_ids = pickle.load(open(CATEGORIES_PKL, "rb"))

In [3]:
result_ids=np.array([], dtype=int)
result_classes=np.array([], dtype=int)

buffer_pred=np.array([], dtype=np.float32) #buffer to keep temporary predictios
buffer_id=np.array([], dtype=np.float32) # buffers to keep temporary ids

INCLUDE_FLIPPED_IMAGES=True

#result_csv="Inceptionv3pytorch_result.csv"
result_predictions="ensemble.h5"

#result_csv = os.path.join("/home/cvpr/Desktop/Nabin/cdiscount/", result_csv)
result_predictions=os.path.join("/home/cvpr/Desktop/Nabin/cdiscount/ensemble", result_predictions)

In [4]:
import tables
hdf5_file = tables.open_file(result_predictions, mode='a')

In [ ]:
#hdf5_ids = hdf5_file.create_earray(hdf5_file.root, 'product_id', tables.Int32Atom(), shape=(0, 1))
#hdf5_categories = hdf5_file.create_earray(hdf5_file.root, 'category_id', tables.Int32Atom(), shape=(0, 1))
#hdf5_categories.append(category_ids.reshape(5270, 1))


In [ ]:
#hdf5_predictions = hdf5_file.create_earray(hdf5_file.root, 'inecptionV3', tables.Float32Atom(), shape=(0, 5270))
#hdf5_predictions = hdf5_file.create_earray(hdf5_file.root, 'xception', tables.Float32Atom(), shape=(0, 5270))

In [5]:
predictions_mean = hdf5_file.create_earray(hdf5_file.root, 'pytorch_resnet101_mean', 
                                           tables.Float32Atom(), shape=(0, 5270),
                                          expectedrows=1768182)
predictions_median = hdf5_file.create_earray(hdf5_file.root, 'pytorch_resnet101_median', 
                                           tables.Float32Atom(), shape=(0, 5270),
                                          expectedrows=1768182)
predictions_max = hdf5_file.create_earray(hdf5_file.root, 'pytorch_resnet101_max', 
                                           tables.Float32Atom(), shape=(0, 5270),
                                          expectedrows=1768182)

In [6]:
class CDiscountPytorch(Dataset):
    
    def __init__(self, csv_file, img_size=(180, 180)):
        '''
        Parameters:
            csv_file: a csv file with all details about images
            img_size: a tuple (image rows, image columns)
        '''
        super(CDiscountPytorch, self).__init__()
        self.csv_file  = pd.read_csv(csv_file)
        self.img_files = self.csv_file["file_name"].values
        self.ids = self.csv_file["id"].values
        self.img_nums=self.csv_file["img_num"].values
        self.img_size=img_size
        
        
    def __getitem__(self, index):
        filename = self.img_files[index]
        img_id = self.ids[index]
        img_num = self.img_nums[index]
        
        image = cv2.imread(filename, 1)
        image = self.transform(image)  
        return image, img_id, img_num
    
    
    def __len__(self):
        return len(self.img_files)
    
    
    def transform1(self,x, reverse_mode=False):
        '''
        converts RGB values from [0,255] 
        to [0, 1] and vice versa
                Parameter:
            reverse_mode: boolean, default is False
                          False:[0, 255]-->[0, 1] 
                          True: [0, 1]-->[0, 255]
        '''
        if reverse_mode:
            return x * 255.
        else:
            return x/255.
    
    def transform2(self,x, reverse_mode=False):
        '''
        converts RGB values from [0,255] 
        to [-1, 1] and vice versa
        
        Parameter:
            reverse_mode: boolean, default is False
                          False:[0, 255]-->[-1, 1] 
                          True: [-1, 1]-->[0, 255]
        '''
        if reverse_mode:
            return ((x / 2.0)+0.5)*255.0
        else:
            return ((x / 255.0)-0.5)*2.0

        
    
    def transform(self,image):
        mean = [0.485, 0.456, 0.406 ]
        std  = [0.229, 0.224, 0.225 ]
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = image.transpose((2,0,1))
        tensor = torch.from_numpy(image).float().div(255.0)
        tensor[0] = (tensor[0] - mean[0]) / std[0]
        tensor[1] = (tensor[1] - mean[1]) / std[1]
        tensor[2] = (tensor[2] - mean[2]) / std[2]
        return tensor 

In [7]:
cdiscount_test_data=CDiscountPytorch(csv_file=FILELIST_CSV)
dataloader = DataLoader(cdiscount_test_data, batch_size=256, num_workers=16)
data=iter(dataloader)
total_batches = len(data)

In [8]:
# cdiscount data set ----
CDISCOUNT_NUM_CLASSES = 5270
CDISCOUNT_HEIGHT=180
CDISCOUNT_WIDTH =180

#load the newtork

#model_file ='LB=0.69673_se-inc3_00026000_model.pth' #does not give this result
#net = SEInception3(in_shape=(3,CDISCOUNT_HEIGHT,CDISCOUNT_WIDTH),num_classes=CDISCOUNT_NUM_CLASSES)

#model_file ='LB=0.69565_inc3_00075000_model.pth'
#net = Inception3(in_shape=(3,CDISCOUNT_HEIGHT,CDISCOUNT_WIDTH),num_classes=CDISCOUNT_NUM_CLASSES)

#model_file='LB=0.69422_xception_00158000_model.pth'
#net = Xception(in_shape=(3,CDISCOUNT_HEIGHT,CDISCOUNT_WIDTH),num_classes=CDISCOUNT_NUM_CLASSES)

model_file='Resnet_101_00243000_model.pth'
net = ResNet101(in_shape=(3,CDISCOUNT_HEIGHT,CDISCOUNT_WIDTH),num_classes=CDISCOUNT_NUM_CLASSES)


net.load_state_dict(torch.load(model_file))
#net.cuda().eval()
net.cuda()

ResNet101 (
  (layer0): Sequential (
    (0): ConvBn2d (
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  )
  (layer1): Sequential (
    (0): Bottleneck (
      (conv_bn1): ConvBn2d (
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      )
      (conv_bn2): ConvBn2d (
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      )
      (conv_bn3): ConvBn2d (
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
      (downsample): ConvBn2d (
        (conv): Conv2d(64,

In [9]:
def process_buffer(data=None, index=None):
    pred_df = pd.DataFrame(data=data,  index=index, columns= category_ids, dtype=np.float32)
    #merge rows with same index (product id)
    pred_df_mean = pred_df.groupby(pred_df.index).mean()
    pred_df_median = pred_df.groupby(pred_df.index).median()
    pred_df_max = pred_df.groupby(pred_df.index).max()
    
    predictions_mean.append(pred_df_mean.values)
    predictions_median.append(pred_df_median.values)
    predictions_max.append(pred_df_max.values)
    #hdf5_ids.append(pred_df.index.values.reshape(-1, 1))
    
    #find the column with maximum predicted value
    #pred_df["category_id"] = pred_df.idxmax(axis=1)

    #result_ids = pred_df.index.tolist()
    #result_classes = pred_df["category_id"].values
    #return result_ids, result_classes
    return

In [10]:
#total_batches=200
bar = tqdm_notebook(total=total_batches)
for batch_number in range(total_batches):
    
    current_batch=data.next()
    
    first_image_number=current_batch[2][0]
    
    ids=np.array(current_batch[1].numpy())
    if INCLUDE_FLIPPED_IMAGES:
        ids=np.concatenate((ids,ids), axis=0)
    
    X_data=current_batch[0]
    if INCLUDE_FLIPPED_IMAGES:
        flipped_images=torch.from_numpy(X_data.numpy()[:,:,:,::-1].copy())
        X_data=torch.cat((X_data, flipped_images), 0)
    
    X_data = Variable(X_data,volatile=True).cuda()
    

    ## Predictions of model
    logits = net(X_data)
    probs  = F.softmax(logits)
    predictions  = probs.cpu().data.numpy()
    del logits, probs
    
    #predictions=F.softmax(net(X_data)).cpu().data.numpy()


    # if it is a first batch, simply put predictions in buffer
    if batch_number==0:
        buffer_pred=predictions
        buffer_id=ids

    else:
        #if first image number is zero
        # 1. first process the items of buffer
        if first_image_number == 0:
            process_buffer(data=buffer_pred,  index=buffer_id)
            #res_ids, res_classes=process_buffer(data=buffer_pred,  index=buffer_id)
            #result_ids = np.concatenate((result_ids, res_ids), axis=0)
            #result_classes = np.concatenate((result_classes, res_classes), axis=0)
            
            # 2. Then put only current predictions and ids in the buffers
            buffer_pred=predictions
            buffer_id=ids
        else:
            # if first image number is not zero simply append predictins and ids to buffer
            buffer_pred = np.concatenate((buffer_pred, predictions), axis=0)
            buffer_id = np.concatenate((buffer_id, ids), axis=0)
    
    if batch_number == total_batches-1:
        process_buffer(data=buffer_pred,  index=buffer_id)
        #res_ids, res_classes=process_buffer(data=buffer_pred,  index=buffer_id)
        #result_ids = np.concatenate((result_ids, res_ids), axis=0)
        #result_classes = np.concatenate((result_classes, res_classes), axis=0)
    
    del ids, predictions, X_data, current_batch
    bar.update()
    
hdf5_file.close()

A Jupyter Widget